In [1]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, note, chord, instrument
import random

# --- Global settings ---
GATE_DURATION = 2.0
PENTATONIC = [0, 2, 4, 7, 9]

# --- Helper functions ---

def quantize_to_pentatonic(midi_note):
    base = 60
    rel = midi_note - base
    octave = rel // 12
    semitone = rel % 12
    closest = min(PENTATONIC, key=lambda x: abs(x - semitone))
    return base + octave * 12 + closest

def state_to_bloch_angles(alpha, beta):
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    return theta, phi

def theta_to_note(theta):
    pitch = int((theta / np.pi) * 12) + 60
    return quantize_to_pentatonic(pitch)

def phi_to_note(phi):
    pitch = int(((phi + np.pi) / (2 * np.pi)) * 12) + 72
    return quantize_to_pentatonic(pitch)

def qubit_to_instrument(index):
    instruments = [instrument.Flute(), instrument.Piano(), instrument.Violin(),
                   instrument.Guitar(), instrument.Clarinet(), instrument.Trumpet()]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, circuit):
    temp_circuit = qiskit.QuantumCircuit(circuit.num_qubits)
    qubit_indices = [circuit.qubits.index(qarg) for qarg in qargs]
    temp_circuit.append(gate, qubit_indices)
    return state.evolve(temp_circuit)

# --- Random circuit generator ---
def generate_random_circuit(num_qubits=2, depth=10, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    qc = QuantumCircuit(num_qubits)
    single_qubit_gates = ['h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'id']
    multi_qubit_gates = ['cx', 'cz', 'swap']

    for _ in range(depth):
        if random.random() < 0.7:  # 70% chance single-qubit gate
            gate = random.choice(single_qubit_gates)
            q = random.randint(0, num_qubits - 1)
            if gate in ['rx', 'ry', 'rz']:
                angle = random.uniform(0, 2 * np.pi)
                getattr(qc, gate)(angle, q)
            else:
                getattr(qc, gate)(q)
        else:  # 30% chance multi-qubit
            if num_qubits < 2:
                continue
            q1, q2 = random.sample(range(num_qubits), 2)
            gate = random.choice(multi_qubit_gates)
            getattr(qc, gate)(q1, q2)

        qc.barrier()

    return qc

# --- Convert circuit to music ---
def circuit_to_music_explicit(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta))
        elif inst.name == 'id':
            for i in range(num_qubits):
                qubit_states[i].append('rest')
        else:
            state = apply_gate(state, inst, qargs, circuit)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        for item in qubit_states[i]:
            if item == 'rest':
                r = note.Rest(quarterLength=GATE_DURATION)
                part.append(r)
            else:
                alpha, beta = item
                theta, phi = state_to_bloch_angles(alpha, beta)
                ch = chord.Chord([theta_to_note(theta), phi_to_note(phi)])
                ch.quarterLength = GATE_DURATION
                part.append(ch)
        music_score.append(part)

    return music_score, qubit_states

# --- Generate random quantum circuit and music ---
qc = generate_random_circuit(num_qubits=3, depth=12, seed=42)

print(qc)  # Optional: print the circuit for inspection

music_score, qubit_states = circuit_to_music_explicit(qc)
music_score.show('text')
music_score.write('midi', fp='quantum_random_music.mid')


           ░ ┌───┐ ░     ░ ┌───┐ ░       ░ ┌────────────┐ ░ ┌───┐ ░ »
q_0: ──────░─┤ Z ├─░──X──░─┤ H ├─░───────░─┤ Rz(2.8225) ├─░─┤ H ├─░─»
           ░ └───┘ ░  │  ░ └───┘ ░       ░ └────────────┘ ░ └───┘ ░ »
q_1: ──────░───────░──┼──░───────░───────░────────────────░───────░─»
     ┌───┐ ░       ░  │  ░       ░ ┌───┐ ░                ░       ░ »
q_2: ┤ H ├─░───────░──X──░───────░─┤ H ├─░────────────────░───────░─»
     └───┘ ░       ░     ░       ░ └───┘ ░                ░       ░ »
«                     ░ ┌───┐ ░                ░     ░ ┌────────────┐ ░ 
«q_0: ────────────────░─┤ X ├─░────────────────░─────░─┤ Rz(3.4686) ├─░─
«     ┌─────────────┐ ░ └─┬─┘ ░ ┌────────────┐ ░     ░ └────────────┘ ░ 
«q_1: ┤ Ry(0.97691) ├─░───■───░─┤ Ry(3.7933) ├─░──X──░────────────────░─
«     └─────────────┘ ░       ░ └────────────┘ ░  │  ░                ░ 
«q_2: ────────────────░───────░────────────────░──X──░────────────────░─
«                     ░       ░                ░     ░                ░ 

OSError: [Errno 30] Read-only file system: 'quantum_random_music.mid'